In [1]:
# Imports

import mlflow
import dagshub
import mlflow.sklearn

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [2]:
# Setting up DagsHub

dagshub.init(repo_owner='SushrutGaikwad', repo_name='youtube-comments-analyzer', mlflow=True)

Accessing as SushrutGaikwad

Initialized MLflow to track repo "SushrutGaikwad/youtube-comments-analyzer"

Repository SushrutGaikwad/youtube-comments-analyzer initialized!

# Data

In [3]:
PREPROCESSED_DATA_PATH = "../data/processed/reddit_preprocessed.csv"
df = pd.read_csv(PREPROCESSED_DATA_PATH)
df.dropna(subset=["clean_comment"], inplace=True)
df.shape

(36662, 2)

# Running the experiment

In [4]:
# Setting experiment name

mlflow.set_experiment("Exp 1: BoW vs. TF-IDF")

2025/06/02 21:31:08 INFO mlflow.tracking.fluent: Experiment with name 'Exp 1: BoW vs. TF-IDF' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/b7e3a4fac7d549d693df980dcc550f89', creation_time=1748880068862, experiment_id='2', last_update_time=1748880068862, lifecycle_stage='active', name='Exp 1: BoW vs. TF-IDF', tags={}>

In [7]:
def run_experiment(
    df,
    vectorizer_type,
    ngram_range,
    vectorizer_max_features,
    vectorizer_name
):
    if vectorizer_type == "BoW":
        vectorizer = CountVectorizer(
            ngram_range=ngram_range,
            max_features=vectorizer_max_features
        )
    elif vectorizer_type == "TF-IDF":
        vectorizer = TfidfVectorizer(
            ngram_range=ngram_range,
            max_features=vectorizer_max_features
        )

    X_train, X_test, y_train, y_test = train_test_split(
        df["clean_comment"],
        df["category"],
        test_size=0.2,
        random_state=42,
        stratify=df["category"]
    )
    X_train_vectorized = vectorizer.fit_transform(X_train)
    X_test_vectorized = vectorizer.transform(X_test)

    with mlflow.start_run() as run:
        # Tags
        mlflow.set_tag("mlflow.runName", f"{vectorizer_name}_{ngram_range}_RandomForest")
        mlflow.set_tag("experiment_type", "feature_engineering")
        mlflow.set_tag("model_type", "RandomForestClassifier")

        # Description
        mlflow.set_tag(
            "description",
            f"RandomForest with {vectorizer_name}, ngram_range={ngram_range}, max_features={vectorizer_max_features}."
        )

        # Logging vectorizer parameters
        mlflow.log_param("vectorizer_type", vectorizer_type)
        mlflow.log_param("ngram_range", ngram_range)
        mlflow.log_param("vectorizer_max_features", vectorizer_max_features)

        # Log random forest parameters
        n_estimators = 200
        max_depth = 15
        mlflow.log_param("n_estimators", n_estimators)
        mlflow.log_param("max_depth", max_depth)

        # Initializing and training the model
        model = RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            random_state=42
        )
        model.fit(X_train_vectorized, y_train)

        # Making predictions on the test set and logging metrics
        y_pred = model.predict(X_test_vectorized)

        # Logging accuracy
        accuracy = accuracy_score(
            y_true=y_test,
            y_pred=y_pred
        )
        mlflow.log_metric("accuracy", accuracy)

        # Logging classification report metrics
        classification_rep = classification_report(
            y_true=y_test,
            y_pred=y_pred,
            output_dict=True
        )
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}: {metric}", value)

        # Logging the confusion matrix
        conf_matrix = confusion_matrix(
            y_true=y_test,
            y_pred=y_pred,
        )
        plt.figure(figsize=(8, 6))
        sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title(f"Confusion matrix: {vectorizer_name}, {ngram_range}")
        plt.savefig(
            f"../reports/figures/confusion_matrix_{vectorizer_name}_{ngram_range}.png"
        )
        mlflow.log_artifact(
            f"../reports/figures/confusion_matrix_{vectorizer_name}_{ngram_range}.png"
        )
        plt.close()

        # Logging the model
        mlflow.sklearn.log_model(
            model,
            f"random_forest_model_{vectorizer_name}_{ngram_range}"
        )

In [8]:
ngram_ranges = [
    (1, 1),  # unigrams
    (1, 2),  # bigrams
    (1, 3)   # trigrams
]
max_features = 5000

for ngram_range in ngram_ranges:
    # BoW experiments
    run_experiment(
        df=df,
        vectorizer_type="BoW",
        ngram_range=ngram_range,
        vectorizer_max_features=max_features,
        vectorizer_name="BoW"
    )

    # TF-IDF experiments
    run_experiment(
        df=df,
        vectorizer_type="TF-IDF",
        ngram_range=ngram_range,
        vectorizer_max_features=max_features,
        vectorizer_name="TF-IDF"
    )

2025/06/02 22:06:39 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run BoW_(1, 1)_RandomForest at: https://dagshub.com/SushrutGaikwad/youtube-comments-analyzer.mlflow/#/experiments/2/runs/638c3ef92fe34e28844517509a004591
🧪 View experiment at: https://dagshub.com/SushrutGaikwad/youtube-comments-analyzer.mlflow/#/experiments/2


2025/06/02 22:07:20 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run TF-IDF_(1, 1)_RandomForest at: https://dagshub.com/SushrutGaikwad/youtube-comments-analyzer.mlflow/#/experiments/2/runs/5c15c60aba1b4a4298bc48e86a98eea5
🧪 View experiment at: https://dagshub.com/SushrutGaikwad/youtube-comments-analyzer.mlflow/#/experiments/2


2025/06/02 22:08:01 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


...

🏃 View run TF-IDF_(1, 2)_RandomForest at: https://dagshub.com/SushrutGaikwad/youtube-comments-analyzer.mlflow/#/experiments/2/runs/7016bc0fb1744a75aa639f603b0e4282
🧪 View experiment at: https://dagshub.com/SushrutGaikwad/youtube-comments-analyzer.mlflow/#/experiments/2


2025/06/02 22:09:25 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run BoW_(1, 3)_RandomForest at: https://dagshub.com/SushrutGaikwad/youtube-comments-analyzer.mlflow/#/experiments/2/runs/6e9bf694afcc48cfa9398833145c4348
🧪 View experiment at: https://dagshub.com/SushrutGaikwad/youtube-comments-analyzer.mlflow/#/experiments/2


2025/06/02 22:10:10 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run TF-IDF_(1, 3)_RandomForest at: https://dagshub.com/SushrutGaikwad/youtube-comments-analyzer.mlflow/#/experiments/2/runs/c046f8e2bd5a48bcaed8dffb3470be9b
🧪 View experiment at: https://dagshub.com/SushrutGaikwad/youtube-comments-analyzer.mlflow/#/experiments/2
